## COGS 182 Homework 5
Malachi Mabie 
10 Feb 2021

###Q1 (2 points)
Exercise 5.4 The pseudocode for Monte Carlo ES is inefficient because, for each state-action pair, it maintains a list of all returns and repeatedly calculates their mean. It would be more efficient to use techniques similar to those explained in Section 2.4 to maintain just the mean and a count (for each state-action pair) and update them incrementally. Describe how the pseudocode would be altered to achieve this.

To update a mean without retaining list of historical data, you would initialize count and update avg per step as:
```
avg = (avg*count + return)/(++count)
```



In exercise 5.4, the relevant pseudocode is:


```
  Append G to Returns(S_t, A_t)
  Q(S_t, A_t) <- average(Returns(S_t, A_t))
```
Instead, we can do:

```
  N[S,A] += 1 // update combination frequency
  Q[S, A] += G - Q[S,A] / N[S,A]
```





Instead of a large list of returns, we add a variable that counts the number of states and actions: ```1/N(new experience - old avg).``` 



###Q2 (4 points)
Use the Monte Carlo Exploring Starts algorithm from page 99 to generate a policy and value functions for the gridworld of example 4.1. Display the value function and policy after the 10, 100, 300, and 1000 episodes. Does the algorithm converge to an optimal policy? [You don't need to worry about making the policy display pretty. You can use matrices to show the probability of each action.]

In [1]:
import numpy as np

transitions = np.array([[0,0,0,0],[0,1,2,5],[1,2,3,6],[2,3,3,7],
                        [4,0,5,8],[4,1,6,9],[5,2,7,10],[6,3,7,11],
                        [8,4,9,12],[8,5,10,13],[9,6,11,14],[10,7,11,0],
                        [12,8,13,12],[12,9,14,13],[13,10,0,14],[0,0,0,0]])

In [2]:
num_states = transitions.shape[0]
values = np.zeros(num_states)
print_at = [10,100,300,1000,10000]

# exploring starts means initialize policy randomly
policy = np.random.randint(low=0,high=3,size=num_states)

# q vals and state action counts start at 0
qvalues = np.zeros([num_states, 4])
sa_count = np.zeros([num_states, 4])

for i in range(1,10001):
  state = np.random.randint(0,15) # allows it to find endzone initially
  action = policy[state]

  ep_sa = []
  ep_reward = []

  # generate an episode - limit loops to prevent infinite
  for step in range(12):
    # encode state-action pair
    ep_sa.append(10*state + action) # unique bc actions are single digit

    if state == 0:
      ep_reward.append(0) # endzone
      break
    ep_reward.append(-1) # constant neg reward

    next_state = transitions[state][action] # get next

    # if next_state == 0:
    #   break # for some reason when i don't add this, some edge values get stuck

    state = next_state
    action = policy[state]

  G = 0
  # loop backwards through episode
  for i_ep, sa in enumerate(reversed(ep_sa)):
    # unencode indice of state-action
    sa_indice = len(ep_reward) - i_ep - 1
    G += ep_reward[sa_indice]
    # unencode state and action from sa
    state = sa//10
    action = sa%10

    # did sa occur earlier in ep?
    if sa not in ep_sa[:sa_indice]:
      sa_count[state][action] += 1
      # perform the avg rewards trick:
      qvalues[state][action] += (G - qvalues[state][action]) / sa_count[state][action]
      # update policy by best action that could have been taken within that state
      policy[state] = np.argmax(qvalues[state])

  # print requested iterations
  if i in print_at:
    print('k = ',i)
    print(np.round(np.max(qvalues,axis=1),1).reshape(4,4))
    

k =  10
[[  0.   0.   0. -12.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]]
k =  100
[[  0.   -1.   -5.4  -8.7]
 [ -1.   -2.3  -3.5  -4.7]
 [ -2.4  -3.   -6.   -1. ]
 [-11.7  -5.1  -1.    0. ]]
k =  300
[[  0.   -1.   -4.4  -6.6]
 [ -1.   -2.1  -3.2  -4.2]
 [ -2.1  -3.   -5.   -1. ]
 [-11.9  -4.5  -1.    0. ]]
k =  1000
[[  0.   -1.   -4.1  -5.6]
 [ -1.   -2.   -3.   -4.1]
 [ -2.   -3.   -4.2  -1. ]
 [-12.   -4.2  -1.    0. ]]
k =  10000
[[  0.   -1.   -4.   -5.1]
 [ -1.   -2.   -3.   -4. ]
 [ -2.   -3.   -4.   -1. ]
 [-12.   -4.   -1.    0. ]]


(Notice how it relies on biased memories, so some equal paths are weighted with preference for one over the other. However, the latest run didn't give anything severe outlying penalties, so it converged to an equanimous solution.)

###Q3 (2 points)
Suppose that the agent was forced to always start in state 6. What changes would you need to make in order to still have the same convergence properties? [You do not need to code the change.]



> If it always starts from state 6, the current max policy would be effected by the extra state 6 experiences. Changing it to epsilon-greedy would solve this problem.



###Q4 (2 points)
Suppose that you are using an off-policy Monte Carlo algorithm with importance sampling. You observe two state-action trajectories both starting from state, $s_0$. Both trajectories had the same probability under the target policy $\pi$. However, the first trajectory had a low probability of occurring under the behavior policy, $b$ and resulted in a $G_t=10$. The second trajectory had a high probability of occurring under the behavior policy, $b$ and resulted in a $G_t=0$. What can we say about the estimate, $v_\pi(s_0)$? Why?



> We want to get $\mathbb{E}[\rho_{t:0} G_t | S_t=s]=v_{\pi}(s)$. The importance ratio is determined by comparing the likelihood of the first trajectory in the behavior policy and the target policy. Probability in pi is always 0.5, and probability in b for step 0 is "low". 0.5/"low" is going to be a large weight, while the second step has 0.5/"high" and therefore a lower weight. So, the expected value of step 0 will be very similar to the behavior policy of trajectory 1.



$$
\rho_{t:T-1}\dot{=}\frac{\prod_{k=t}^{T-1}\pi(A_k|S_k)p(S_{k+t}|S_k,A_k}{\prod_{k=t}^{T-1}b(A_k|S_k)p(S_{k+t}|S_k,A_k} = \prod_{k=t}^{T-1}\frac{\pi(A_k|S_k)}{b(A_k|S_k)}\\\\
V(s) \dot{=}\frac{\sum_{t\in\tau(s)}\rho_{t:T(t)-1}G_t}{\sum_{t\in\tau(s)}\rho_{t:T(t)-1}} = \rho(10)+\rho(0)
$$